In [86]:
import pandas as pd

In [87]:
df = pd.read_csv('data.csv')

In [88]:
df.head()

,Distance in Kilometres,Ahmedabad,Bangalore,Bhubaneshwar,Bombay,Calcutta,Chandigarh,Cochin,Delhi,Hyderabad,...,Jaipur,Kanpur,Lucknow,Madras,Nagpur,Nasik,Panjim,Patna,Pondicherry,Pune
0,Agartala,3305,3824,2286,3593,1863,2998,4304,2708,3330,...,2801,2281,2252,3493,2696,3365,3507,1681,3661,3442
1,Agra,878,1848,1578,1202,1300,448,2278,200,1246,...,230,290,369,2048,770,1005,1715,885,2210,1214
2,Ahmedabad,-,1490,1697,552,2068,1157,1845,911,1436,...,648,1168,1247,1821,965,504,1165,1656,1818,664
3,Allahabad,1251,1686,1090,1457,817,912,2216,650,1084,...,713,193,234,2011,608,1155,1419,402,1077,1364
4,Amritsar,1356,2496,2224,1849,1919,239,3163,445,1892,...,706,926,939,2688,1416,1665,2237,1531,2856,1862


In [89]:
# assertions to check validity of data
for i in range(1, len(df.columns)):
    assert (df.columns[i] in df['Distance in Kilometres'].values)
    cities = df['Distance in Kilometres']
    dist = df[df.columns[i]]
    for j in range(len(dist)):
        if (cities[j] not in df.columns):
            continue
        dist_1 = dist[j]
        dist_2 = df.iloc[j][df.columns[i]]
        assert (dist_1 == dist_2)

In [90]:
# seperating class 1 cities in a new dataframe
df_ = {'Distance in Kilometres': list(df.columns[1:])}
for city in df['Distance in Kilometres']:
    df_[city] = []
for from_city in df.columns[1:]:
    for idx in range(len(df[from_city])):
        to_city_dist = df[from_city][idx]
        df_[df['Distance in Kilometres'][idx]].append(to_city_dist)
df_ = pd.DataFrame(df_)

,Distance in Kilometres,Agartala,Agra,Ahmedabad,Allahabad,Amritsar,Asansol,Bangalore,Baroda,Bhopal,...,Pondicherry,Pune,Ranchi,Shillong,Shimla,Surat,Trivandrum,Varanasi,Vijayawada,Vishakapatnam
0,Ahmedabad,3305,878,-,1251,1356,1842,1490,119,523,...,1818,664,1781,2839,1256,263,2197,1373,1705,1815
1,Bangalore,3824,1848,1490,1686,2496,2187,-,1408,1148,...,328,826,2098,3242,2395,1264,696,1791,637,1093
2,Bhubaneshwar,2286,1578,1697,1090,2224,523,1538,1604,1162,...,1375,1587,560,1640,2127,1579,1953,968,775,445
3,Bombay,3593,1202,552,1457,1849,2040,1013,433,778,...,1452,184,1816,3011,1753,301,1376,1597,973,1754
4,Calcutta,1863,1300,2068,817,1919,226,1961,1937,1495,...,1798,2058,414,1281,1839,1912,2376,695,1198,868
5,Chandigarh,2998,448,1157,912,239,1503,2296,1181,988,...,2657,1663,1480,2416,99,1325,3051,1032,1959,2127
6,Cochin,4304,2278,1845,2216,3163,2544,512,1763,1995,...,530,1181,2455,3599,3056,1593,218,2331,1174,1449
7,Delhi,2708,200,911,650,445,1262,2049,1151,742,...,2411,1417,1214,2126,365,1182,1950,745,1716,1881
8,Hyderabad,3330,1246,1436,1084,1892,1693,563,1127,808,...,867,545,1434,1309,1812,983,1261,1189,269,599
9,Indore,2891,591,442,803,1258,1394,1601,379,191,...,1963,623,1333,2309,1171,607,2156,925,1255,1433


In [91]:
# remove class 1 from class 2
for i in range(1, len(df.columns)):
    city = df.columns[i]
    idx = df['Distance in Kilometres'][df['Distance in Kilometres'] == city].index[0]
    df.drop(idx, inplace=True)

,Distance in Kilometres,Ahmedabad,Bangalore,Bhubaneshwar,Bombay,Calcutta,Chandigarh,Cochin,Delhi,Hyderabad,...,Jaipur,Kanpur,Lucknow,Madras,Nagpur,Nasik,Panjim,Patna,Pondicherry,Pune
0,Agartala,3305,3824,2286,3593,1863,2998,4304,2708,3330,...,2801,2281,2252,3493,2696,3365,3507,1681,3661,3442
1,Agra,878,1848,1578,1202,1300,448,2278,200,1246,...,230,290,369,2048,770,1005,1715,885,2210,1214
3,Allahabad,1251,1686,1090,1457,817,912,2216,650,1084,...,713,193,234,2011,608,1155,1419,402,1077,1364
4,Amritsar,1356,2496,2224,1849,1919,239,3163,445,1892,...,706,926,939,2688,1416,1665,2237,1531,2856,1862
5,Asansol,1842,2187,523,2040,226,1503,2544,1262,1693,...,1304,789,825,1857,1122,1746,2300,395,2024,1955
7,Baroda,119,1408,1604,433,1937,1181,1763,1151,1127,...,789,1230,1311,1739,774,457,1158,1582,1735,545
8,Bhopal,523,1148,1162,778,1495,988,1995,742,808,...,596,585,664,1604,352,605,1143,1016,1772,814
12,Calicut,1648,520,1923,1171,2346,2741,222,2494,910,...,2523,2260,2339,715,1483,1193,576,2476,566,984
15,Coimbatore,1669,333,1633,1192,2057,2669,195,2412,912,...,2369,2218,2297,426,1441,1214,800,2434,410,1005
17,Gwalior,919,1734,1486,1085,1224,462,1881,315,1132,...,351,280,359,1928,656,891,1467,809,2096,1100


In [92]:
print('Class 1 cities')
print(df_)
print('Class 2 cities')
print(df)

Class 1 cities
   Distance in Kilometres Agartala  Agra Ahmedabad Allahabad Amritsar Asansol  \
0               Ahmedabad     3305   878         -      1251     1356    1842   
1               Bangalore     3824  1848      1490      1686     2496    2187   
2            Bhubaneshwar     2286  1578      1697      1090     2224     523   
3                  Bombay     3593  1202       552      1457     1849    2040   
4                Calcutta     1863  1300      2068       817     1919     226   
5              Chandigarh     2998   448      1157       912      239    1503   
6                  Cochin     4304  2278      1845      2216     3163    2544   
7                   Delhi     2708   200       911       650      445    1262   
8               Hyderabad     3330  1246      1436      1084     1892    1693   
9                  Indore     2891   591       442       803     1258    1394   
10                 Jaipur     2801   230       648       713      706    1304   
11           

In [94]:
df_.to_csv('class_1_cities.csv', index=False)
df.to_csv('class_2_cities.csv', index=False)